In [1]:
import cv2 , os

In [21]:
data_path="C:/Users/hp/Desktop/face-mask-detection-keras-master/dataset"
categories= os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'with mask': 0, 'without mask': 1}
['with mask', 'without mask']
[0, 1]


In [22]:
img_size=100
data=[]
target=[]
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        
        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        
        except Exception as e:
            print('Exception:',e)
            

In [23]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)


In [24]:
np.save("data",data)
np.save("target",new_target)

In [25]:
import numpy as np

data=np.load("data.npy")
target=np.load("target.npy")

In [26]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [28]:

checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Train on 997 samples, validate on 250 samples
Epoch 1/20
997/997 [==============================] - 43s 43ms/step - loss: 0.7153 - accuracy: 0.5567 - val_loss: 0.6482 - val_accuracy: 0.6400
Epoch 2/20
997/997 [==============================] - 44s 44ms/step - loss: 0.5475 - accuracy: 0.7352 - val_loss: 0.5775 - val_accuracy: 0.7040
Epoch 3/20
997/997 [==============================] - 45s 45ms/step - loss: 0.3870 - accuracy: 0.8295 - val_loss: 0.3731 - val_accuracy: 0.8520
Epoch 4/20
997/997 [==============================] - 44s 44ms/step - loss: 0.2916 - accuracy: 0.8847 - val_loss: 0.2868 - val_accuracy: 0.9080
Epoch 5/20
997/997 [==============================] - 44s 44ms/step - loss: 0.1957 - accuracy: 0.9268 - val_loss: 0.2320 - val_accuracy: 0.9160
Epoch 6/20
997/997 [==============================] - 46s 46ms/step - loss: 0.1389 - accuracy: 0.9549 - val_loss: 0.2894 - val_accuracy: 0.8880
Epoch 7/20
997/997 [==============================] - 46s 46ms/step - loss: 0.1436 - accur

In [29]:
print(model.evaluate(test_data,test_target))

139/139 [==============================] - 1s 10ms/step
[0.08754037524298798, 0.9640287756919861]


In [30]:

from keras.models import load_model
import cv2
import numpy as np

In [31]:

model = load_model('model-008.model')

In [32]:

face_clsfr=cv2.CascadeClassifier('C:/Users/hp/Desktop/face-mask-detection-keras-master/haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [33]:

while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()